In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pickle
import seaborn as sns
import time
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from numpy import concatenate
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.semi_supervised import LabelPropagation
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB 
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
import pyspark
import random
import pandas as pd
import sqlite3
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('C_Large-2.csv')
c = sqlite3.connect('jupyter_sql_tutorial.db')

In [3]:
#df.to_sql('Q1', c)

In [4]:
%load_ext sql

%sql sqlite:///jupyter_sql_tutorial.db

In [10]:
%%sql
DROP TABLE Q1


 * sqlite:///jupyter_sql_tutorial.db
Done.


[]

In [11]:
%%sql
DROP TABLE Q2

 * sqlite:///jupyter_sql_tutorial.db
Done.


[]

In [7]:
df.head()

,Id,QueryString,TopClickedUrls,PopularityRank,PredictedLabel
0,df5cf101-85df-4fda-8450-21b5ef82c1c1,cs1513 c# } expected,https://stackoverflow.com/questions/5394409/co...,1,False
1,ad3de02b-dabe-4b3d-bba0-1541ad214935,cs2001 c# source file could not be found.,https://stackoverflow.com/questions/19406924/e...,2,False
2,80049825-dd92-4fd0-a073-d9b12247aa83,cs1001 c# identifier expected,https://stackoverflow.com/questions/4006164/er...,3,False
3,dfb425e4-737e-416d-a362-4323235b5c1c,ide0055 c# fix formatting,https://marketplace.visualstudio.com/items?ite...,4,False
4,5aeb856b-d0b0-4260-a8a5-ff947ab80431,cs7036 c# there is no argument given that corr...,https://stackoverflow.com/questions/54555468/c...,5,False


In [7]:
def cleanText(text):
    
    lemma = WordNetLemmatizer()
    stp = stopwords.words('english')
    
    text = re.sub("[^a-zA-Z0-9#]"," ",text)
    text = text.lower()

    text = nltk.word_tokenize(text)
    text = [lemma.lemmatize(word) for word in text]
    
    text = [word for word in text if word not in stp]
    
    text = " ".join(text)
    
    return text

In [8]:
cleanedText = []
for text in df["QueryString"]:   
    cleanedText.append(cleanText(text))
print(cleanedText[0])

cs1513 c # expected


In [13]:
y = df["PredictedLabel"]
y.shape

(1010261,)

In [12]:
dt={'id': df["Id"], 'query': cleanedText, 'rank': df["PopularityRank"], 'urls': df["TopClickedUrls"], 'label': y}
df1 = pd.DataFrame(dt)
df1.head()

,id,query,rank,urls,label
0,df5cf101-85df-4fda-8450-21b5ef82c1c1,cs1513 c # expected,1,https://stackoverflow.com/questions/5394409/co...,False
1,ad3de02b-dabe-4b3d-bba0-1541ad214935,cs2001 c # source file could found,2,https://stackoverflow.com/questions/19406924/e...,False
2,80049825-dd92-4fd0-a073-d9b12247aa83,cs1001 c # identifier expected,3,https://stackoverflow.com/questions/4006164/er...,False
3,dfb425e4-737e-416d-a362-4323235b5c1c,ide0055 c # fix formatting,4,https://marketplace.visualstudio.com/items?ite...,False
4,5aeb856b-d0b0-4260-a8a5-ff947ab80431,cs7036 c # argument given corresponds required...,5,https://stackoverflow.com/questions/54555468/c...,False


In [14]:
df1["len"] = [len(text) for text in df1["query"].values]

In [15]:
df1.head()

,id,query,rank,urls,label,len
0,df5cf101-85df-4fda-8450-21b5ef82c1c1,cs1513 c # expected,1,https://stackoverflow.com/questions/5394409/co...,False,19
1,ad3de02b-dabe-4b3d-bba0-1541ad214935,cs2001 c # source file could found,2,https://stackoverflow.com/questions/19406924/e...,False,34
2,80049825-dd92-4fd0-a073-d9b12247aa83,cs1001 c # identifier expected,3,https://stackoverflow.com/questions/4006164/er...,False,30
3,dfb425e4-737e-416d-a362-4323235b5c1c,ide0055 c # fix formatting,4,https://marketplace.visualstudio.com/items?ite...,False,26
4,5aeb856b-d0b0-4260-a8a5-ff947ab80431,cs7036 c # argument given corresponds required...,5,https://stackoverflow.com/questions/54555468/c...,False,63


In [16]:
df1.to_sql('Q1', c)

In [18]:
%%time
%%sql
SELECT *
FROM Q1
WHERE len = 20 AND label = False AND query ='c # vb net converter'

 * sqlite:///jupyter_sql_tutorial.db
Done.
Wall time: 723 ms


index,id,query,rank,urls,label,len
784,ea08dad2-ccb0-4e07-a577-a16ad6d629a3,c # vb net converter,419,"http://converter.telerik.com/,https://converter.telerik.com/,https://www.developerfusion.com/tools/convert/csharp-to-vb/",0,20
1179,98e4a0c5-1825-4633-8ca9-fce123ef447c,c # vb net converter,473,"https://converter.telerik.com/,http://converter.telerik.com/,https://www.developerfusion.com/tools/convert/csharp-to-vb/",0,20
5758,ea08dad2-ccb0-4e07-a577-a16ad6d629a3,c # vb net converter,419,"http://converter.telerik.com/,https://converter.telerik.com/,https://www.developerfusion.com/tools/convert/csharp-to-vb/",0,20
6153,98e4a0c5-1825-4633-8ca9-fce123ef447c,c # vb net converter,473,"https://converter.telerik.com/,http://converter.telerik.com/,https://www.developerfusion.com/tools/convert/csharp-to-vb/",0,20
10732,ea08dad2-ccb0-4e07-a577-a16ad6d629a3,c # vb net converter,419,"http://converter.telerik.com/,https://converter.telerik.com/,https://www.developerfusion.com/tools/convert/csharp-to-vb/",0,20
11127,98e4a0c5-1825-4633-8ca9-fce123ef447c,c # vb net converter,473,"https://converter.telerik.com/,http://converter.telerik.com/,https://www.developerfusion.com/tools/convert/csharp-to-vb/",0,20
15706,ea08dad2-ccb0-4e07-a577-a16ad6d629a3,c # vb net converter,419,"http://converter.telerik.com/,https://converter.telerik.com/,https://www.developerfusion.com/tools/convert/csharp-to-vb/",0,20
16101,98e4a0c5-1825-4633-8ca9-fce123ef447c,c # vb net converter,473,"https://converter.telerik.com/,http://converter.telerik.com/,https://www.developerfusion.com/tools/convert/csharp-to-vb/",0,20
20680,ea08dad2-ccb0-4e07-a577-a16ad6d629a3,c # vb net converter,419,"http://converter.telerik.com/,https://converter.telerik.com/,https://www.developerfusion.com/tools/convert/csharp-to-vb/",0,20
21075,98e4a0c5-1825-4633-8ca9-fce123ef447c,c # vb net converter,473,"https://converter.telerik.com/,http://converter.telerik.com/,https://www.developerfusion.com/tools/convert/csharp-to-vb/",0,20


In [20]:
indexed_df= df1.set_index(['label', 'len'])
sorted_ix_df=indexed_df.sort_index()
sorted_ix_df

id  \
label len                                         
False 3    e529061d-94ea-4a8f-b7ce-d00c4b3e05a2   
      3    aa5f504c-658d-45a7-8dec-addc31ebb0e8   
      3    b0c3b761-c517-46ba-8851-b6e6e4bf613e   
      3    e529061d-94ea-4a8f-b7ce-d00c4b3e05a2   
      3    aa5f504c-658d-45a7-8dec-addc31ebb0e8   
...                                         ...   
True  116  035ec89b-6b79-4022-a7c6-d71781ee3b8e   
      116  035ec89b-6b79-4022-a7c6-d71781ee3b8e   
      116  035ec89b-6b79-4022-a7c6-d71781ee3b8e   
      116  035ec89b-6b79-4022-a7c6-d71781ee3b8e   
      116  035ec89b-6b79-4022-a7c6-d71781ee3b8e   

                                                       query  rank  \
label len                                                            
False 3                                                  c #   560   
      3                                                  c #   562   
      3                                                  c #   569   
      3                                                  c #   560   
      3                                                  c #   562   
...                                                      ...   ...   
True  116  cs1069 c # type name registrykey could found n...   564   
      116  cs1069 c # type name registrykey could found n...   564   
      116  cs1069 c # type name registrykey could found n...   564   
      116  cs1069 c # type name registrykey could found n...   564   
      116  cs1069 c # type name registrykey could found n...   564   

                                                        urls  
label len                                                     
False 3    http://www.tutorialsteacher.com/csharp/csharp-...  
      3    https://docs.microsoft.com/en-us/dotnet/csharp...  
      3    https://stackoverflow.com/questions/2848184/or...  
      3    http://www.tutorialsteacher.com/csharp/csharp-...  
      3    https://docs.microsoft.com/en-us/dotnet/csharp...  
...                                                      ...  
True  116  https://docs.microsoft.com/en-us/dotnet/api/mi...  
      116  https://docs.microsoft.com/en-us/dotnet/api/mi...  
      116  https://docs.microsoft.com/en-us/dotnet/api/mi...  
      116  https://docs.microsoft.com/en-us/dotnet/api/mi...  
      116  https://docs.microsoft.com/en-us/dotnet/api/mi...  

[1010261 rows x 4 columns]

In [21]:
sorted_ix_df.to_sql('Q2', c)

In [22]:
%reload_ext sql

%sql sqlite:///jupyter_sql_tutorial.db

In [28]:
%%time
%%sql
SELECT *
FROM Q2
WHERE len = 20 AND label = False AND query ='c # vb net converter'

 * sqlite:///jupyter_sql_tutorial.db
Done.
Wall time: 6.99 ms


label,len,id,query,rank,urls
0,20,ea08dad2-ccb0-4e07-a577-a16ad6d629a3,c # vb net converter,419,"http://converter.telerik.com/,https://converter.telerik.com/,https://www.developerfusion.com/tools/convert/csharp-to-vb/"
0,20,98e4a0c5-1825-4633-8ca9-fce123ef447c,c # vb net converter,473,"https://converter.telerik.com/,http://converter.telerik.com/,https://www.developerfusion.com/tools/convert/csharp-to-vb/"
0,20,ea08dad2-ccb0-4e07-a577-a16ad6d629a3,c # vb net converter,419,"http://converter.telerik.com/,https://converter.telerik.com/,https://www.developerfusion.com/tools/convert/csharp-to-vb/"
0,20,98e4a0c5-1825-4633-8ca9-fce123ef447c,c # vb net converter,473,"https://converter.telerik.com/,http://converter.telerik.com/,https://www.developerfusion.com/tools/convert/csharp-to-vb/"
0,20,ea08dad2-ccb0-4e07-a577-a16ad6d629a3,c # vb net converter,419,"http://converter.telerik.com/,https://converter.telerik.com/,https://www.developerfusion.com/tools/convert/csharp-to-vb/"
0,20,98e4a0c5-1825-4633-8ca9-fce123ef447c,c # vb net converter,473,"https://converter.telerik.com/,http://converter.telerik.com/,https://www.developerfusion.com/tools/convert/csharp-to-vb/"
0,20,ea08dad2-ccb0-4e07-a577-a16ad6d629a3,c # vb net converter,419,"http://converter.telerik.com/,https://converter.telerik.com/,https://www.developerfusion.com/tools/convert/csharp-to-vb/"
0,20,98e4a0c5-1825-4633-8ca9-fce123ef447c,c # vb net converter,473,"https://converter.telerik.com/,http://converter.telerik.com/,https://www.developerfusion.com/tools/convert/csharp-to-vb/"
0,20,ea08dad2-ccb0-4e07-a577-a16ad6d629a3,c # vb net converter,419,"http://converter.telerik.com/,https://converter.telerik.com/,https://www.developerfusion.com/tools/convert/csharp-to-vb/"
0,20,98e4a0c5-1825-4633-8ca9-fce123ef447c,c # vb net converter,473,"https://converter.telerik.com/,http://converter.telerik.com/,https://www.developerfusion.com/tools/convert/csharp-to-vb/"
